In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from collections import OrderedDict

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from keybert import KeyBERT

In [3]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import matplotlib.pyplot as plt
import spacy
import string
from sklearn.metrics.pairwise import cosine_similarity

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
#list(gensim.downloader.info()['models'].keys())
#wv = api.load('glove-wiki-gigaword-300')
#wv.save('glove_wiki_vectors.kv')

In [5]:
revalive_path_vec = "../../glove_wiki_vectors.kv"
relative_path_sdg = "../../sdg_data.csv"
relative_path_attr = "../Dictionary/Attributes_2015-16.csv"
relative_string_map = "./Data_Goal/String_map.csv"
relative_string_map_unroll = "./Data_Goal/String_map_unroll.csv"
relative_semantic = "./Data_Goal/Semantic_map.csv"
relative_semantic_unroll = "./Data_Goal/Semantic_map_unroll.csv"
relative_sem_threshold = "./Data_Goal/Semantic_threshold_map.csv"
relative_sem_threshold_unroll = "./Data_Goal/Semantic_threshold_map_unroll.csv"

# Get the absolute path by joining the current directory with the relative path
absolute_path_vec = os.path.normpath(os.path.join(os.getcwd(), revalive_path_vec))
absolute_path_sdg = os.path.normpath(os.path.join(os.getcwd(), relative_path_sdg))
absolute_path_attr = os.path.normpath(os.path.join(os.getcwd(), relative_path_attr))
absolute_path_string = os.path.normpath(os.path.join(os.getcwd(), relative_string_map))
absolute_path_string_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_string_map_unroll))
absolute_path_semantic = os.path.normpath(os.path.join(os.getcwd(), relative_semantic))
absolute_path_semantic_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_semantic_unroll))
absolute_path_sem_threshold =os.path.normpath( os.path.join(os.getcwd(), relative_sem_threshold))
absolute_path_sem_threshold_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_sem_threshold_unroll))

In [6]:
wv = KeyedVectors.load(absolute_path_vec)

In [7]:
kw_model = KeyBERT(model=wv)

In [8]:
def replace_words(df, column_list, replace_dict):
    for col in column_list:
        # Apply the replace function to the specified column using the replace_dict
        df[col] = df[col].replace(replace_dict, regex=True)
    return df

# Define the replacement dictionary
replace_dict = {
    'B.p.l.': 'below poverty line',
    'A.p.l.': 'above poverty line',
    'Bpl': 'below poverty line',
}


In [9]:
IGNORE_WORDS = ['proportion', 'whose', 'number', 'per', 'within', 'no', 'under', 'volume', 'use', 'type', 'percentage', 'total', 'data', 'year', 'due', 'rate', 'hects', 'hect', 'noof', 'years']

In [10]:
# Creating our tokenizer function
def cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()
    ignore_word = []
    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        #letters_list = list(set(letters))
        letters_list = list(OrderedDict.fromkeys(letters))   
        cleaned.append(letters_list)

    return cleaned

In [11]:
# Creating our tokenizer function
def concat_cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()

    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        #letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        #IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        #letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        letters = ' '.join(letters)
        cleaned.append(letters)

    return cleaned

In [12]:
def process_keyword(data):
    result = []
    for sublist in data:
        inner_list = []
        for word, _ in sublist:
            # Split the word if it contains a space and reverse the order of words
            words = word.split()
            # Filter out numeric values
            words = [w for w in words if not w.isdigit()]
            if words:  # Exclude empty lists
                inner_list.append(words)
        inner_list.reverse()
        result.append(inner_list)
    return result

In [13]:
def add_elements(row):
    if not row['keywords']:  # Check if col2 is empty
        row['keywords'].extend([[elem] for elem in row['new_goal']])  # Add elements from col1 to col2 as lists
    return row

In [14]:
def concatenate_table_name(df):
    for index, row in df.iterrows():
        if ',' not in row['Description']:
            df.at[index, 'Description'] += ', ' + row['Table_name']
        elif row['Description'].count(',') == 1:
            parts = row['Description'].split(',')
            if len(parts) == 2:
                part1, part2 = parts
                if re.match(r'^\w+$', part1.strip()) or re.match(r'^\w+$', part2.strip()):
                    df.at[index, 'Description'] += ', ' + row['Table_name']
    return df


In [15]:
def concatenate_all(df):
    for index, row in df.iterrows():
        df.at[index, 'Description'] += ', ' + row['Table_name'] + ', ' + row['Chapter_name'] 
    return df

In [16]:
missing_words = []
miss_added = []
def sent_vec(row):
    sent = row['new_description']
    cap_tab = row['Chapter_name'] + ", " + row['Table_name']
    vector_size = wv.vector_size
    vectors = []
    miss_w = []
    for w in sent:
        if w in wv:
            wv_res = np.zeros(vector_size)
            wv_res = wv[w]
            vectors.append(wv_res)
        else:
            missing_words.append(("KAG", w))
            miss_w.append(w)
    if len(miss_w)>0:
        vectors.append(missin_word_vec(cap_tab))
        miss_added.append(miss_w)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [17]:
def missin_word_vec(sent):
    vector_size = wv.vector_size
    sent_list = cleaning([sent])
    wv_res = np.zeros(vector_size)
    ctr = 0
    for w in sent_list[0]:
        if w in wv:
            wv_res += wv[w]
            ctr += 1
    if ctr>0:
        wv_res = wv_res/ctr
    else:
        wv_res = np.random.rand(vector_size)
        print('yes')
    return wv_res

In [18]:
def sent_vec_avg(sent):
    vector_size = wv.vector_size
    missing_words = []
    vectors = []
    for w in sent:
        ctr = 0
        wv_res = np.zeros(vector_size)
        for word in  w:
            if word in wv:
                wv_res += wv[word]
                ctr += 1
            else:
                missing_words.append(word)
        if ctr>0:
            wv_res = wv_res/ctr
        vectors.append(wv_res)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [19]:
def count_common_words(row, df_other):
    set_1 = set(row)
    common_words = []
    for _, other_row in df_other.iterrows():
        set_2 = set(other_row['new_description'])
        common_words.append(len(set_1.intersection(set_2)))
    return common_words

In [20]:
def find_similarity_string(similarity_matrix, threshold, chapter_df, sdg_df):
    top_n_similarities = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_chapter_name = []
    top_n_table_id = []
    top_n_table_name = []
    top_n_count = []
    top_n_description = []
    
    for index, row in similarity_matrix.iterrows():
        # Filter indices based on threshold value
        top_n_indices = [i for i, value in enumerate(row) if value >= threshold]
        
        # Sort indices based on similarity values
        top_n_indices = sorted(top_n_indices, key=lambda i: row.iloc[i], reverse=True)
        
        # Get top n values, chapters, and chapter names
        values = [row.iloc[i] for i in top_n_indices]
        attr = [similarity_matrix.columns[i] for i in top_n_indices]
        attr_des = [chapter_df['Description'].loc[i] for i in top_n_indices]
        chapter_id = [chapter_df['Chapter_id'].loc[i] for i in top_n_indices]
        chapter_name = [chapter_df['Chapter_name'].loc[i] for i in top_n_indices]   
        table_id = [chapter_df['Table_id'].loc[i] for i in top_n_indices]
        table_name = [chapter_df['Table_name'].loc[i] for i in top_n_indices]   


        # Append to respective lists
        top_n_count.append(len(attr))
        top_n_similarities.append(values)
        top_n_attr.append(attr)
        top_n_description.append(attr_des)
        top_n_chapter_id.append(chapter_id)
        top_n_chapter_name.append(chapter_name)
        top_n_table_id.append(table_id)
        top_n_table_name.append(table_name)
    
    # Assign lists to the SDG DataFrame
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_table_name'] = top_n_table_name
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_similarities'] = top_n_similarities
    sdg_df['top_n_description'] = top_n_description

    return sdg_df


In [21]:
def process_result_string(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        values = row['top_n_similarities']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        
        for attr, chap, tab, val, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, values, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no,
                'top_n_similarities': val
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'top_n_goal_id', 'top_n_similarities']]
    return all_goals


In [22]:
def find_similarity_semantic(sdg_df, att_df, threshold):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_similarity_values = []
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        top_min_list = []
        for i in ind_list[ind]:
            max_list = []
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_similarity = np.max(similarity_matrix)
                max_list.append(max_similarity)
            # Get the indices of the top 10 max values
            #top_indices = [idx for idx, sim in enumerate(max_list) if sim < 1]
            top_indices = sorted(range(len(max_list)), key=lambda i: max_list[i], reverse=True)[:threshold]
            top_indices_list.append(top_indices)
            top_sim = [max_list[i] for i in top_indices]
            top_sim_list.append(top_sim)
            top_min_list.append(min(top_sim))
            
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
            
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        min_similarity_values.append(top_min_list)
    return min_similarity_values, top_indices_values


In [23]:
#indicator words to attribute complete name
def find_similarity_semantic_threshold(sdg_df, att_df, diff):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_list = sdg_df['min_values'].tolist()
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        min_values = min_list[ind]
        for index, i in enumerate(ind_list[ind]):
            max_list = []
            threshold = min_values[index] - diff
            if(threshold <= 0):
                continue
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_value = np.max(similarity_matrix)
                max_list.append((max_value, att))
            
            # Get the indices of values strictly greater than threshold
            top_indices = [idx for val, idx in max_list if val > threshold]
            # Sort top_indices based on the corresponding values in max_list
            top_indices.sort(key=lambda x: next((val for val, idx in max_list if idx == x), 0), reverse=True)
            top_indices_list.append(top_indices)
            top_sim = [max_list[i][0] for i in top_indices]
            top_sim_list.append(top_sim)
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
        
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        
    return top_indices_values

In [24]:
def finding_attr_details(sdg_df, att_df):
    result_list = sdg_df['top_n_index'].tolist()
    top_n_count = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_table_id = []
    top_n_description = []
    top_n_chapter_name = []
    top_n_table_name = []
    for ll in result_list:
        top_n_count.append(len(ll))
        top_n_attr.append([att_df.loc[idx, 'Attr_id'] for idx in ll])
        top_n_chapter_id.append([att_df.loc[idx, 'Chapter_id'] for idx in ll])
        top_n_table_id.append([att_df.loc[idx, 'Table_id'] for idx in ll])
        top_n_description.append([att_df.loc[idx, 'Description'] for idx in ll])
        top_n_chapter_name.append([att_df.loc[idx, 'Chapter_name'] for idx in ll])
        top_n_table_name.append([att_df.loc[idx, 'Table_name'] for idx in ll])
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_description'] = top_n_description
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_table_name'] = top_n_table_name
    return sdg_df

In [25]:
def process_result_semantic(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        
        for attr, chap, tab, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'top_n_goal_id']]
    return all_goals


In [26]:
sdg = pd.read_csv(absolute_path_sdg, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [27]:
sdg_complete = sdg.copy()

In [28]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [29]:
sdg.columns

Index(['Goal No.', 'Goal', 'Nodal Department', 'Targets',
       'Other Related Major Departments', 'Tentative Indicators', 'Target_id',
       'Indicator_id'],
      dtype='object')

In [30]:
sdg.drop(['Nodal Department','Other Related Major Departments', 'Tentative Indicators', 'Indicator_id', 'Targets', 'Target_id'],axis=1,inplace=True)

In [31]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [32]:
sdg.reset_index(inplace = True, drop = True)

In [33]:
sdg

,Goal No.,Goal
0,1,End poverty in all its forms everywhere
1,2,"End hunger, achieve food security and improved..."
2,3,Ensure healthy lives and promote well-being fo...
3,4,Ensure inclusive and equitable quality educati...
4,5,Achieve gender equality and empower all women ...
5,6,Ensure availability and sustainable management...
6,7,"Ensure access to affordable, reliable, sustain..."
7,8,"Promote sustained, inclusive and sustainable e..."
8,9,"Build resilient infrastructure, promote inclus..."
9,10,Reduce inequality within the State


In [34]:
sdg_new = sdg.copy()

In [35]:
texto_data=sdg_new['Goal'].tolist()
process_text=cleaning(texto_data)

cleaned_sent = pd.DataFrame({'new_goal': process_text})
sdg_new['new_goal']=cleaned_sent

In [36]:
texto_data0=sdg_new['Goal'].tolist()
process_text0=concat_cleaning(texto_data0)

cleaned_sent0 = pd.DataFrame({'new_goal_str': process_text0})
sdg_new['new_goal_str']=cleaned_sent0

In [37]:
sdg_new

,Goal No.,Goal,new_goal,new_goal_str
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state


In [38]:
data = pd.read_csv(absolute_path_attr, sep=";", dtype={'Attr_id': str, 'Chapter_id': str, 'Table_id':str})

In [39]:
# Replace words in the DataFrame using the dictionary
data = replace_words(data, ['Chapter_name', 'Table_name', 'Description'], replace_dict)

In [40]:
data_new = concatenate_all(data)
data_new

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ..."
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Circles, Nada Offices Village Accountant Circl..."
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl..."
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun..."
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ..."
...,...,...,...,...,...,...
1162,1179.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag..."
1163,1180.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric..."
1164,1181.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr..."
1165,1182.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota..."


In [41]:
# Create a new DataFrame with selected columns
new_df = data_new.copy()
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ..."
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Circles, Nada Offices Village Accountant Circl..."
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl..."
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun..."
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ..."
...,...,...,...,...,...,...
1162,1179.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag..."
1163,1180.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric..."
1164,1181.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr..."
1165,1182.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota..."


In [42]:
texto_data1=new_df['Description'].tolist()
process_text1=cleaning(texto_data1)

cleaned_sent1 = pd.DataFrame({'new_description': process_text1})
new_df['new_description']=cleaned_sent1

In [43]:
texto_data11=new_df['Description'].tolist()
process_text11=concat_cleaning(texto_data11)

cleaned_sent11 = pd.DataFrame({'new_description_str': process_text11})
new_df['new_description_str']=cleaned_sent11

In [44]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ...","[nada, office, village, accountant, circle, ho...",nada offices nada offices village accountant c...
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Circles, Nada Offices Village Accountant Circl...","[circle, nada, office, village, accountant, ho...",circles nada offices village accountant circle...
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl...","[hoblies, nada, office, village, accountant, c...",hoblies nada offices village accountant circle...
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun...","[grama, panchayaths, nada, office, village, ac...",grama panchayaths nada offices village account...
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ...","[taluks, nada, office, village, accountant, ci...",noof taluks nada offices village accountant ci...
...,...,...,...,...,...,...,...,...
1162,1179.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag...","[agrl, land, holder, agricultural, holding, ar...",total number total agrl land holder total agri...
1163,1180.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric...","[male, area, agrl, land, holder, agricultural,...",male area total agrl land holder total agricul...
1164,1181.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr...","[female, area, agrl, land, holder, agricultura...",female area total agrl land holder total agric...
1165,1182.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota...","[institution, area, agrl, land, holder, agricu...",institutions area total agrl land holder total...


String Matching

In [45]:
common_word_matrix = sdg_new['new_goal'].apply(count_common_words, args=(new_df,))
common_word_matrix

0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
10    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
11    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
12    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
13    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
14    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
15    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
16    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: new_goal, dtype: object

In [46]:
attr_id = new_df['Attr_id'].tolist()
ind_id = sdg_new['Goal No.'].tolist()

In [47]:
# Determine the maximum number of words in any list
max_words = max(common_word_matrix, key=lambda x: len(x))

# Create DataFrame with columns Word_1, Word_2, ..., Word_n
common_word_df = pd.DataFrame(common_word_matrix.tolist(), columns=attr_id, index=ind_id)

In [48]:
common_word_df

,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,...,1174.0,1175.0,1176.0,1177.0,1178.0,1179.0,1180.0,1181.0,1182.0,1183.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Example usage:
threshold_value = 1  # Define the threshold value

# Call the function to find chapter_ids with similarity greater than the threshold
sdg_string_map = find_similarity_string(common_word_df, threshold_value, new_df, sdg_new.copy())


In [50]:
sdg_string_map

,Goal No.,Goal,new_goal,new_goal_str,top_n_count,top_n_attr,top_n_table_id,top_n_table_name,top_n_chapter_id,top_n_chapter_name,top_n_similarities,top_n_description
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,18,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.7, 1.7, 1.7, ...",[Ration Shops And below poverty line Ration Ca...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General Information, General Information, Gen...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Urban,Ration shops, Ration Shops And below po..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,210,"[207.0, 240.0, 294.0, 156.0, 157.0, 158.0, 159...","[4.3, 4.6, 4.12, 4.1, 4.1, 4.1, 4.1, 4.1, 4.1,...","[Area under principal crops Year 2014-15, Prod...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[Agriculture, Horticulture and Sericulture, Ag...","[2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Total Food Grains,pulses Hect,Area under prin..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,26,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.1...",[Districtwise Population By Age Groups And Sex...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, ...","[AREA AND POPULATION, AREA AND POPULATION, ARE...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Male,0-14, Districtwise Population By Age Gro..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,114,"[534.0, 535.0, 536.0, 537.0, 538.0, 539.0, 540...","[10.1, 10.1, 10.1, 10.1, 10.1, 10.1, 10.1, 10....","[Literates per 2011 Census, Literates per 2011...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[Education, Education, Education, Education, E...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Male,Rural,Literates, Literates per 2011 Cens..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,82,"[863.0, 555.0, 559.0, 563.0, 576.0, 579.0, 583...","[13.6, 10.3, 10.4, 10.5, 10.8, 10.8, 10.9, 10....",[No.of Beneficiaries under different schemes Y...,"[13, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[Women Child Development, Education, Education...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[No.of benefitted for payment of Girl children...
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,25,"[25.0, 524.0, 588.0, 598.0, 883.0, 884.0, 885....","[1.5, 9.3, 10.9, 10.10, 14.4, 14.4, 14.4, 14.4...","[No.of Fire Stations on 31-03-2016 In Nos, Rur...","[1, 9, 10, 10, 14, 14, 14, 14, 14, 14, 14, 14,...","[General Information, Transport and Communicat...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Working Fire Brigade Water Tankers, No.of Fir..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,28,"[523.0, 524.0, 525.0, 526.0, 527.0, 528.0, 905...","[9.3, 9.3, 9.3, 9.3, 9.3, 9.3, 15.2, 16.1, 16....",[Rural Roads Length and Number of Habitations ...,"[9, 9, 9, 9, 9, 9, 15, 16, 16, 16, 16, 16, 16,...","[Transport and Communication, Transport and Co...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Pakka,Panchayat Roads,Rural Roads Length on 3..."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,9,"[389.0, 416.0, 418.0, 868.0, 869.0, 879.0, 880...","[5.4, 6.

In [51]:
string_all_goals = process_result_string(sdg_string_map, sdg_new)

In [52]:
sdg_string_map.to_csv(absolute_path_string, sep=";", index=False)

In [53]:
string_all_goals.to_csv(absolute_path_string_unroll, sep=";", index=False)

Semantic Mapping

In [54]:
sdg_sem = sdg_new.copy()

In [55]:
word_count = sdg_sem['new_goal'].apply(lambda x: len(x))
word_count

0      4
1     10
2      6
3      9
4      6
5      6
6      7
7     11
8     10
9      3
10     8
11     5
12     7
13    17
14    14
15     8
16     2
Name: new_goal, dtype: int64

In [56]:
from collections import Counter
w_c = Counter(word_count)
w_c.most_common()

[(6, 3),
 (10, 2),
 (7, 2),
 (8, 2),
 (4, 1),
 (9, 1),
 (11, 1),
 (3, 1),
 (5, 1),
 (17, 1),
 (14, 1),
 (2, 1)]

In [57]:
w_c.most_common(10)

[(6, 3),
 (10, 2),
 (7, 2),
 (8, 2),
 (4, 1),
 (9, 1),
 (11, 1),
 (3, 1),
 (5, 1),
 (17, 1)]

In [58]:
median_indicator = np.median(word_count)
print(median_indicator)

7.0


In [59]:
from statistics import mode
mode_indicator = mode(word_count)
mode_indicator

6

In [60]:
np.mean(word_count)

7.823529411764706

In [61]:
stopwords_set = list(set(nltk.corpus.stopwords.words('english')))
stopwords_set.extend(IGNORE_WORDS)
stopwords_set = list(set(stopwords_set))
stopwords_set

['year',
 're',
 'were',
 'proportion',
 'y',
 'within',
 'his',
 'when',
 'm',
 "it's",
 'total',
 'its',
 "weren't",
 'but',
 'hasn',
 'there',
 "that'll",
 'number',
 "shan't",
 'too',
 'haven',
 'have',
 'just',
 'which',
 'same',
 'can',
 'who',
 'herself',
 'i',
 'here',
 'having',
 'itself',
 'our',
 'all',
 "didn't",
 "mightn't",
 "aren't",
 'hadn',
 've',
 'ain',
 'down',
 'other',
 "you'd",
 'ma',
 "hasn't",
 'being',
 'your',
 'to',
 'very',
 'about',
 'how',
 "you'll",
 'at',
 'my',
 'wasn',
 "doesn't",
 'out',
 'so',
 "don't",
 'after',
 'themselves',
 's',
 "mustn't",
 'he',
 'years',
 "hadn't",
 'me',
 'their',
 "haven't",
 'rate',
 'if',
 'through',
 'again',
 'or',
 "shouldn't",
 'myself',
 'mightn',
 'am',
 'why',
 "needn't",
 'won',
 'no',
 'of',
 'yourself',
 'above',
 'noof',
 'each',
 't',
 'this',
 'should',
 'him',
 'between',
 'than',
 'couldn',
 "won't",
 "wouldn't",
 "you're",
 'them',
 'such',
 'into',
 'now',
 'volume',
 'wouldn',
 'ours',
 'll',
 'weren',


In [62]:
#using 'new_indicators_str', takin top_n=6 as median = 8 and nr_candidates=10 as mean = 9.2
sdg_sem['keywords']=kw_model.extract_keywords(sdg_sem['new_goal_str'].tolist(), keyphrase_ngram_range=(1,2), stop_words = stopwords_set, use_maxsum=True, nr_candidates=10, top_n=6)

In [63]:
sdg_sem['keywords'] = process_keyword(sdg_sem['keywords'].tolist())
sdg_sem

,Goal No.,Goal,new_goal,new_goal_str,keywords
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge..."
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,"[[modern, energy], [ensure], [affordable, reli..."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,"[[full, productive], [promote, sustained], [de..."
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...,"[[promote], [inclusive, sustainable], [foster,..."
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state,[]


In [64]:
# Apply the function to each row of the DataFrame
sdg_sem = sdg_sem.apply(add_elements, axis=1)
sdg_sem

,Goal No.,Goal,new_goal,new_goal_str,keywords
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge..."
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,"[[modern, energy], [ensure], [affordable, reli..."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,"[[full, productive], [promote, sustained], [de..."
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...,"[[promote], [inclusive, sustainable], [foster,..."
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state,"[[reduce], [inequality], [state]]"


In [65]:
sdg_sem['vec'] = sdg_sem['keywords'].apply(sent_vec_avg)
sdg_sem

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [...","[[0.004565000534057617, 0.3802250027656555, 0...."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv...","[[0.07902299799025059, 0.040918700862675905, -..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h...","[[-0.13508999347686768, 0.3590700030326843, 0...."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l...","[[-0.15599000453948975, -0.4886299967765808, -..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge...","[[-0.4389500021934509, 0.47641998529434204, 0...."
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m...","[[0.2156050056219101, -0.03203999996185303, 0...."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,"[[modern, energy], [ensure], [affordable, reli...","[[0.4583650156855583, -0.1715994980186224, -0...."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,"[[full, productive], [promote, sustained], [de...","[[-0.29617999494075775, 0.21565200434997678, -..."
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...,"[[promote], [inclusive, sustainable], [foster,...","[[0.0451820008456707, 0.06997399777173996, -0...."
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state,"[[reduce], [inequality], [state]]","[[0.21908999979496002, 0.35475999116897583, 0...."


In [66]:
new_df['vec'] = new_df.apply(sent_vec, axis=1)

In [67]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str,vec
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ...","[nada, office, village, accountant, circle, ho...",nada offices nada offices village accountant c...,"[[-0.07122399657964706, -0.3697200119495392, -..."
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Circles, Nada Offices Village Accountant Circl...","[circle, nada, office, village, accountant, ho...",circles nada offices village accountant circle...,"[[0.15791000425815582, 0.24285000562667847, -0..."
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl...","[hoblies, nada, office, village, accountant, c...",hoblies nada offices village accountant circle...,"[[-0.07122399657964706, -0.3697200119495392, -..."
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun...","[grama, panchayaths, nada, office, village, ac...",grama panchayaths nada offices village account...,"[[0.09846500307321548, -0.4200499951839447, -0..."
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ...","[taluks, nada, office, village, accountant, ci...",noof taluks nada offices village accountant ci...,"[[-0.18463000655174255, -1.0946999788284302, -..."
...,...,...,...,...,...,...,...,...,...
1162,1179.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag...","[agrl, land, holder, agricultural, holding, ar...",total number total agrl land holder total agri...,"[[-0.13323000073432922, -0.3314499855041504, -..."
1163,1180.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric...","[male, area, agrl, land, holder, agricultural,...",male area total agrl land holder total agricul...,"[[-0.324180006980896, 0.4799000024795532, -0.1..."
1164,1181.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr...","[female, area, agrl, land, holder, agricultura...",female area total agrl land holder total agric...,"[[-0.22990000247955322, 0.4291200041770935, -0..."
1165,1182.0,18,Additional information,18.3,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota...","[institution, area, agrl, land, holder, agricu...",institutions area total agrl land holder total...,"[[-0.035638000816106796, -0.29197999835014343,..."


In [68]:
missing_words_set = list(set(missing_words))
print(len(missing_words_set), missing_words_set)

86 [('KAG', 'tubectomy'), ('KAG', 'nddp'), ('KAG', 'cultur'), ('KAG', 'tbcontrol'), ('KAG', 'prosy'), ('KAG', 'nrlm'), ('KAG', 'districtwise'), ('KAG', 'kutera'), ('KAG', 'primar'), ('KAG', 'referal'), ('KAG', 'allopathy'), ('KAG', 'rsin'), ('KAG', 'omini'), ('KAG', 'sectorwise'), ('KAG', 'tbpatients'), ('KAG', 'gistered'), ('KAG', 'hepatities'), ('KAG', 'estp'), ('KAG', 'rsinlakhs'), ('KAG', 'bengalgram'), ('KAG', 'granate'), ('KAG', 'agril'), ('KAG', 'ariculated'), ('KAG', 'agrl'), ('KAG', 'mtonnes'), ('KAG', 'multiaxled'), ('KAG', 'numberof'), ('KAG', 'avare'), ('KAG', 'heeps'), ('KAG', 'sesamum'), ('KAG', 'lpcd'), ('KAG', 'nogovt'), ('KAG', 'eterinary'), ('KAG', 'famalies'), ('KAG', 'raods'), ('KAG', 'sterlisation'), ('KAG', 'intons'), ('KAG', 'beneficiries'), ('KAG', 'intonnes'), ('KAG', 'mandays'), ('KAG', 'ashraya'), ('KAG', 'jananni'), ('KAG', 'sqkms'), ('KAG', 'anthyodaya'), ('KAG', 'mgnregs'), ('KAG', 'suraksha'), ('KAG', 'facilitiesin'), ('KAG', 'inyears'), ('KAG', 'rainguag

In [69]:
miss_added_set = list(set([item for sublist in miss_added for item in sublist]))
print(len(miss_added_set), miss_added_set)

86 ['noofbenefited', 'indigenious', 'cultur', 'prosy', 'agril', 'avare', 'anthyodaya', 'referal', 'suraksh', 'mtonnes', 'ckward', 'nddp', 'mahathma', 'vehicals', 'suraksha', 'rainguage', 'beneficiries', 'intonnes', 'bhagyajyoti', 'mandays', 'rikshaws', 'undermulberry', 'estp', 'sqkms', 'sesamum', 'benificiaries', 'gistered', 'guarentee', 'rainfallfrom', 'eterinary', 'selfemployment', 'numberof', 'lpcd', 'bengalgram', 'intons', 'kscard', 'rsinlakhs', 'nogovt', 'sthers', 'granate', 'ipsets', 'gddp', 'sectorwise', 'rainydays', 'primar', 'jananni', 'sapota', 'omini', 'proporationof', 'facilitiesin', 'delivaries', 'raods', 'inyears', 'bhagyalakshmi', 'istyear', 'highwa', 'districtwise', 'handpump', 'anganawadi', 'tbcontrol', 'creeches', 'ashraya', 'sterlisation', 'famalies', 'kutera', 'tubectomy', 'nrlm', 'agrl', 'mgnregs', 'nitrozen', 'yojane', 'multiaxled', 'hepatities', 'emergencyfirecall', 'hoblies', 'allopathy', 'rsin', 'tubewell', 'sandya', 'agricensus', 'mathsasrayascheme', 'brambedk

In [70]:
sdg_count_threshold = sdg_sem.copy()

In [71]:
min_val_list, top_n_index_list = find_similarity_semantic(sdg_count_threshold, new_df, 10)

In [72]:
sdg_count_threshold['top_n_index'] = top_n_index_list
sdg_count_threshold['min_values'] = min_val_list
sdg_count_threshold

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec,top_n_index,min_values
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [...","[[0.004565000534057617, 0.3802250027656555, 0....","[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 381, ...","[0.8735124280076773, 0.5514384505494149, 0.512..."
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv...","[[0.07902299799025059, 0.040918700862675905, -...","[199, 232, 286, 418, 419, 148, 150, 151, 152, ...","[0.8411997113893511, 0.5815403183778527, 0.566..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h...","[[-0.13508999347686768, 0.3590700030326843, 0....","[169, 170, 171, 172, 173, 174, 882, 883, 903, ...","[0.7045711183911056, 0.6798916546022344, 0.527..."
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l...","[[-0.15599000453948975, -0.4886299967765808, -...","[526, 527, 529, 530, 534, 535, 536, 538, 539, ...","[1.0000000000000002, 0.41478819432196234, 0.50..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge...","[[-0.4389500021934509, 0.47641998529434204, 0....","[594, 869, 547, 551, 555, 568, 571, 575, 585, ...","[0.6183490921829581, 0.411797604971018, 0.4003..."
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m...","[[0.2156050056219101, -0.03203999996185303, 0....","[22, 516, 580, 590, 876, 877, 878, 879, 880, 8...","[0.5454543688980285, 0.5270143992244825, 1.000..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",ensure access to affordable reliable sustainab...,"[[modern, energy], [ensure], [affordable, reli...","[[0.4583650156855583, -0.1715994980186224, -0....","[515, 516, 517, 518, 519, 520, 898, 930, 931, ...","[0.8285528169164933, 0.5270143992244825, 0.471..."
7,8,"Promote sustained, inclusive and sustainable e...","[promote, sustained, inclusive, sustainable, e...",promote sustained inclusive and sustainable ec...,"[[full, productive], [promote, sustained], [de...","[[-0.29617999494075775, 0.21565200434997678, -...","[408, 410, 861, 862, 872, 873, 874, 875, 381, ...","[0.5318692418751239, 0.5479780674858141, 0.703..."
8,9,"Build resilient infrastructure, promote inclus...","[build, resilient, infrastructure, promote, in...",build resilient infrastructure promote inclusi...,"[[promote], [inclusive, sustainable], [foster,...","[[0.0451820008456707, 0.06997399777173996, -0....","[573, 574, 575, 576, 577, 578, 579, 580, 581, ...","[0.5658785348581976, 0.4676713963912552, 0.526..."
9,10,Reduce inequality within the State,"[reduce, inequality, state]",reduce inequality within the state,"[[reduce], [inequality], [state]]","[[0.21908999979496002, 0.35475999116897583, 0....","[512, 832, 833, 834, 835, 836, 53, 54, 49, 50,...","[0.5343018032154383, 0.5327294572224268, 0.533..."


In [73]:
sdg_min_value = sdg_count_threshold.copy()

In [74]:
sdg_count_threshold_all = finding_attr_details(sdg_count_threshold, new_df)
sdg_count_threshold_all

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec,top_n_index,min_values,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [...","[[0.004565000534057617, 0.3802250027656555, 0....","[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 381, ...","[0.8735124280076773, 0.5514384505494149, 0.512...",37,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 3, 3, 5, ...","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.7, 1.7, 1.7, ...","[Urban,Ration shops, Ration Shops And below po...","[General Information, General Information, Gen...",[Ration Shops And below poverty line Ration Ca...
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv...","[[0.07902299799025059, 0.040918700862675905, -...","[199, 232, 286, 418, 419, 148, 150, 151, 152, ...","[0.8411997113893511, 0.5815403183778527, 0.566...",58,"[207.0, 240.0, 294.0, 426.0, 427.0, 156.0, 158...","[4, 4, 4, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4.3, 4.6, 4.12, 6.4, 6.4, 4.1, 4.1, 4.1, 4.1,...","[Total Food Grains,pulses Hect,Area under prin...","[Agriculture, Horticulture and Sericulture, Ag...","[Area under principal crops Year 2014-15, Prod..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h...","[[-0.13508999347686768, 0.3590700030326843, 0....","[169, 170, 171, 172, 173, 174, 882, 883, 903, ...","[0.7045711183911056, 0.6798916546022344, 0.527...",46,"[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[4, 4, 4, 4, 4, 4, 14, 14, 15, 9, 10, 11, 11, ...","[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 14.5, 14.5, 15....","[Nos,Wells,2014-15 Net area irrigated Hectares...","[Agriculture, Horticulture and Sericulture, Ag...",[Gross and Net area Irrigated under Different ...
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l...","[[-0.15599000453948975, -0.4886299967765808, -...","[526, 527, 529, 530, 534, 535, 536, 538, 539, ...","[1.0000000000000002, 0.41478819432196234, 0.50...",54,"[534.0, 535.0, 537.0, 538.0, 542.0, 543.0, 544...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 1...","[10.1, 10.1, 10.1, 10.1, 10.1, 10.2, 10.2, 10....","[Male,Rural,Literates, Literates per 2011 Cens...","[Education, Education, Education, Education, E...","[Literates per 2011 Census, Literates per 2011..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge...","[[-0.4389500021934509, 0.47641998529434204, 0....","[594, 869, 547, 551, 555, 568, 571, 575, 585, ...","[0.6183490921829581, 0.411797604971018, 0.4003...",55,"[602.0, 876.0, 555.0, 559.0, 563.0, 576.0, 579...","[10, 14, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[10.11, 14.3, 10.3, 10.4, 10.5, 10.8, 10.8, 10...","[Men,No. of Lecturers, No.of P U Colleges Stud...","[Education, Rural Development & Panchayat Raj,...",[No.of P U Colleges Students and Lecturers Yea...
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m...","[[0.2156050056219101, -0.03203999996185303, 0....","[22, 516, 580, 590, 876, 877, 878, 879, 880, 8...","[0.5454543688980285, 0.5270143992244825, 1.000...",43,"[25.0, 524.0, 588.0, 598.0, 883.0, 884.0, 885....","[1, 9, 10, 10, 14, 14, 14, 14, 14, 14, 9, 9, 9..."

In [75]:
sdg_copy = sdg_sem.copy()

In [76]:
sdg_count_threshold_all_goals = process_result_semantic(sdg_count_threshold_all, sdg_copy)

In [77]:
sdg_count_threshold_all.to_csv(absolute_path_semantic, sep=";", index=False)

In [78]:
sdg_count_threshold_all_goals.to_csv(absolute_path_semantic_unroll, sep=";", index=False)

In [79]:
sdg_count_threshold_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,top_n_goal_id
0,28.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Urban,Ration shops, Ration Shops And below pov...",End poverty in all its forms everywhere,1
1,29.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Rural,Ration shops, Ration Shops And below pov...",End poverty in all its forms everywhere,1
2,30.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Total,Ration shops, Ration Shops And below pov...",End poverty in all its forms everywhere,1
3,31.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Urban,Anthyodaya,below poverty line Card Holde...",End poverty in all its forms everywhere,1
4,32.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Rural,Anthyodaya,below poverty line Card Holde...",End poverty in all its forms everywhere,1
...,...,...,...,...,...,...,...,...
740,539.0,10,Education,10.1,Literates per 2011 Census,"Total,Urban,Literates, Literates per 2011 Cens...","Data, monitoring and accountability",17
741,540.0,10,Education,10.1,Literates per 2011 Census,"Male,Total,Literates, Literates per 2011 Censu...","Data, monitoring and accountability",17
742,541.0,10,Education,10.1,Literates per 2011 Census,"Female,Total,Literates, Literates per 2011 Cen...","Data, monitoring and accountability",17
743,542.0,10,Education,10.1,Literates per 2011 Census,"Total,Literates, Literates per 2011 Census, Ed...","Data, monitoring and accountability",17


In [80]:
sdg_count_sim = sdg_sem.copy()

In [81]:
top_index = find_similarity_semantic_threshold(sdg_min_value, new_df, 0.025)

In [82]:
sdg_count_sim['top_n_index'] = top_index

In [83]:
sdg_count_sim_all = finding_attr_details(sdg_count_sim, new_df)
sdg_count_sim_all

,Goal No.,Goal,new_goal,new_goal_str,keywords,vec,top_n_index,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"[end, poverty, form, everywhere]",end poverty in all its forms everywhere,"[[end, poverty], [end], [forms, everywhere], [...","[[0.004565000534057617, 0.3802250027656555, 0....","[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 3...",75,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.7, 1.7, 1.7, ...","[Urban,Ration shops, Ration Shops And below po...","[General Information, General Information, Gen...",[Ration Shops And below poverty line Ration Ca...
1,2,"End hunger, achieve food security and improved...","[end, hunger, achieve, food, security, improve...",end hunger achieve food security and improved ...,"[[sustainable, agriculture], [security, improv...","[[0.07902299799025059, 0.040918700862675905, -...","[199, 232, 286, 418, 419, 148, 150, 151, 152, ...",362,"[207.0, 240.0, 294.0, 426.0, 427.0, 156.0, 158...","[4, 4, 4, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4.3, 4.6, 4.12, 6.4, 6.4, 4.1, 4.1, 4.1, 4.1,...","[Total Food Grains,pulses Hect,Area under prin...","[Agriculture, Horticulture and Sericulture, Ag...","[Area under principal crops Year 2014-15, Prod..."
2,3,Ensure healthy lives and promote well-being fo...,"[ensure, healthy, life, promote, well, age]",ensure healthy lives and promote well being fo...,"[[well], [well, ages], [ensure], [promote], [h...","[[-0.13508999347686768, 0.3590700030326843, 0....","[169, 170, 171, 172, 173, 174, 882, 883, 903, ...",186,"[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[4, 4, 4, 4, 4, 4, 14, 14, 15, 9, 9, 9, 9, 9, ...","[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 14.5, 14.5, 15....","[Nos,Wells,2014-15 Net area irrigated Hectares...","[Agriculture, Horticulture and Sericulture, Ag...",[Gross and Net area Irrigated under Different ...
3,4,Ensure inclusive and equitable quality educati...,"[ensure, inclusive, equitable, quality, educat...",ensure inclusive and equitable quality educati...,"[[education], [ensure, inclusive], [promote, l...","[[-0.15599000453948975, -0.4886299967765808, -...","[526, 527, 529, 530, 534, 535, 536, 538, 539, ...",341,"[534.0, 535.0, 537.0, 538.0, 542.0, 543.0, 544...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[10.1, 10.1, 10.1, 10.1, 10.1, 10.2, 10.2, 10....","[Male,Rural,Literates, Literates per 2011 Cens...","[Education, Education, Education, Education, E...","[Literates per 2011 Census, Literates per 2011..."
4,5,Achieve gender equality and empower all women ...,"[achieve, gender, equality, empower, woman, girl]",achieve gender equality and empower all women ...,"[[women], [equality, empower], [equality], [ge...","[[-0.4389500021934509, 0.47641998529434204, 0....","[594, 869, 547, 551, 555, 568, 571, 575, 585, ...",388,"[602.0, 876.0, 555.0, 559.0, 563.0, 576.0, 579...","[10, 14, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[10.11, 14.3, 10.3, 10.4, 10.5, 10.8, 10.8, 10...","[Men,No. of Lecturers, No.of P U Colleges Stud...","[Education, Rural Development & Panchayat Raj,...",[No.of P U Colleges Students and Lecturers Yea...
5,6,Ensure availability and sustainable management...,"[ensure, availability, sustainable, management...",ensure availability and sustainable management...,"[[ensure, availability], [ensure], [water], [m...","[[0.2156050056219101, -0.03203999996185303, 0....","[22, 516, 580, 590, 876, 877, 878, 879, 880, 8...",211,"[25.0, 524.0, 588.0, 598.0, 883.0, 884.0, 885....","[1, 9, 10, 10, 14, 14, 14, 14, 14, 14, 14, 14,...","[1.5, 9.3, 10.9, 10.10, 14.4, 14.4, 14.4, 14.4...","[Working Fire Brigade Water Tankers, No.of Fir...","[General Information, Transport and Communicat...","[No.of Fire Stations on 31-03-2016 In Nos, Rur..."
6,7,"Ensure access to affordable, reliable, sustain...","[ensure, access, affordable, reliable, sustain...",e

In [84]:
sdg_copy2 = sdg_sem.copy()

In [85]:
sdg_count_sim_all_goals = process_result_semantic(sdg_count_sim_all, sdg_copy2)

In [86]:
sdg_count_sim_all.to_csv(absolute_path_sem_threshold, sep=";", index=False)

In [87]:
sdg_count_sim_all_goals.to_csv(absolute_path_sem_threshold_unroll, sep=";", index=False)

In [88]:
sdg_count_sim_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,top_n_goal_id
0,28.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Urban,Ration shops, Ration Shops And below pov...",End poverty in all its forms everywhere,1
1,29.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Rural,Ration shops, Ration Shops And below pov...",End poverty in all its forms everywhere,1
2,30.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Total,Ration shops, Ration Shops And below pov...",End poverty in all its forms everywhere,1
3,31.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Urban,Anthyodaya,below poverty line Card Holde...",End poverty in all its forms everywhere,1
4,32.0,1,General Information,1.6,Ration Shops And below poverty line Ration Car...,"Rural,Anthyodaya,below poverty line Card Holde...",End poverty in all its forms everywhere,1
...,...,...,...,...,...,...,...,...
3842,632.0,10,Education,10.14,No.of Polytechnic Colleges and Students Year- ...,"No.of Institutions,Polytechnic- Government, No...","Data, monitoring and accountability",17
3843,633.0,10,Education,10.14,No.of Polytechnic Colleges and Students Year- ...,"Male,Istyear,Polytechnic- Government, No.of Po...","Data, monitoring and accountability",17
3844,634.0,10,Education,10.14,No.of Polytechnic Colleges and Students Year- ...,"Female,Istyear,Polytechnic- Government, No.of ...","Data, monitoring and accountability",17
3845,635.0,10,Education,10.14,No.of Polytechnic Colleges and Students Year- ...,"Total,Istyear,Polytechnic- Government, No.of P...","Data, monitoring and accountability",17
